In [1]:
import pandas as pd
import sys
sys.path.append("/home/acomajuncosa/Documents/ready-to-screen-enamine-real/src")
from src import smiles_to_ecfp
from src import save_fingerprints
from src import upload
from src import save_ids
import os

In [2]:
SMILES = pd.read_csv("/home/acomajuncosa/Documents/chembl-antimicrobial-tasks/config/chembl_activities/compound_structures.csv")['canonical_smiles'].tolist()

In [3]:
# Calculate and save ECFP4s
OUTPUT_SMILES, X = smiles_to_ecfp(SMILES[:1_000_0])
OUTPUT_SMILES = [[str(c), i] for c, i in enumerate(OUTPUT_SMILES)]

# Save data
save_fingerprints(X, open("../tmp/X.npz", "wb"))
save_ids(OUTPUT_SMILES, open("../tmp/ids.pq", "wb"))

# Upload data
upload("../tmp/X.npz", "../service.json")
upload("../tmp/ids.pq", "../service.json")

# Remove files
os.remove("../tmp/X.npz")
os.remove("../tmp/ids.pq")

100%|██████████| 10000/10000 [00:02<00:00, 4930.88it/s]
